<a href="https://colab.research.google.com/github/willywijayas/C2BANGKIT/blob/main/Code_for_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cosine Similarity for Recommender Systems 

## Import Data

In [ ]:
import os 
import tempfile 

import numpy as np
import pandas as pd

In [ ]:
# To Display Output without max columns 

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/dummy_data - usaha.csv", delimiter=',')

# developer-friendly columns name
df.columns = [x.lower().replace(' ', '_').replace('-', '_') for x in df.columns.to_list()]

df.sample(1)

,user_id,jenis_usaha,skala_usaha,modal_usaha,bidang_usaha,omset_usaha,usia_targetpelanggan,gender_targetpelanggan,pekerjaan_targetpelanggan,status_targetpelanggan,jenis_lokasi_,unnamed:_11,unnamed:_12,unnamed:_13,unnamed:_14,unnamed:_15,unnamed:_16,unnamed:_17,unnamed:_18,unnamed:_19,unnamed:_20,unnamed:_21,unnamed:_22,unnamed:_23,unnamed:_24,unnamed:_25
147,70010154,Usaha Pijat dan Perawatan Tubuh,Mikro,5-10 JT,Jasa kesehatan dan kecantikan,0-10 JT,"12-17 tahun, 18-39 tahun, 40-59 tahun",Perempuan,"Pelajar/mahasiswa,\nPegawai negeri sipil,\nWir...","Kelas atas, Kelas menengah","Pusat Kota dan Keramaian, Permukiman penduduk,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## EDA 

In [ ]:
df=df[['user_id', 'jenis_usaha', 'skala_usaha', 'modal_usaha', 'bidang_usaha', 'omset_usaha', 'usia_targetpelanggan', 'gender_targetpelanggan', 'pekerjaan_targetpelanggan', 'status_targetpelanggan', 'jenis_lokasi_']]

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183 entries, 0 to 182
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   user_id                    183 non-null    int64 
 1   jenis_usaha                183 non-null    object
 2   skala_usaha                183 non-null    object
 3   modal_usaha                183 non-null    object
 4   bidang_usaha               183 non-null    object
 5   omset_usaha                183 non-null    object
 6   usia_targetpelanggan       183 non-null    object
 7   gender_targetpelanggan     183 non-null    object
 8   pekerjaan_targetpelanggan  183 non-null    object
 9   status_targetpelanggan     183 non-null    object
 10  jenis_lokasi_              183 non-null    object
dtypes: int64(1), object(10)
memory usage: 15.9+ KB


In [ ]:
print(df.duplicated().sum())
print(df.isna().sum())

1
user_id                      0
jenis_usaha                  0
skala_usaha                  0
modal_usaha                  0
bidang_usaha                 0
omset_usaha                  0
usia_targetpelanggan         0
gender_targetpelanggan       0
pekerjaan_targetpelanggan    0
status_targetpelanggan       0
jenis_lokasi_                0
dtype: int64


In [ ]:
df.drop_duplicates(inplace=True)
df.duplicated().sum()

0

## Data Pre-Processing Convert to Numeric 

Note : 
* Data Encoding dan One Hot Encoding () 
* Feature Scaling

STR to LIST

In [ ]:
#Split Item Data for Multiple Choice : 'usia_targetpelanggan', 'gender_targetpelanggan', 'pekerjaan_targetpelanggan', 'status_targetpelanggan', 'jenis_lokasi_'

df['usia_targetpelanggan'] = df['usia_targetpelanggan'].map(lambda x : x.split(","))
df['gender_targetpelanggan'] = df['gender_targetpelanggan'].map(lambda x : x.split(","))
df['pekerjaan_targetpelanggan'] = df['pekerjaan_targetpelanggan'].map(lambda x : x.split(","))
df['status_targetpelanggan'] = df['status_targetpelanggan'].map(lambda x : x.split(","))
df['jenis_lokasi_'] = df['jenis_lokasi_'].map(lambda x : x.split(","))

Feature Encoding

In [ ]:
#Feature encoding untuk feature 'skala_usaha', 'modal_usaha', 'omset_usaha'
mapping_skala = {
    'Mikro' : 1,
    'Kecil' : 2,
    'Menengah' : 3
}

mapping_modal = {
    '<=3 JT':1,
    '3-5 JT' : 2, 
    '5-10 JT': 3, 
    '11-30 JT': 4, 
    '30-50 JT': 5, 
    '50-500 JT':6, 
    '<=1 M': 7
}

mapping_omset = {
    '0-10 JT': 1, 
    '11-20 JT': 2, 
    '21-30 JT': 3, 
    '31-40 JT': 4, 
    '41-50 JT': 5, 
    '>50 JT': 6
}

df['skala_usaha'] = df['skala_usaha'].map(mapping_skala)
df['modal_usaha'] = df['modal_usaha'].map(mapping_modal)
df['omset_usaha'] = df['omset_usaha'].map(mapping_omset)

In [ ]:
df.sample(3)

,id_usaha,jenis_usaha,skala_usaha,modal_usaha,bidang_usaha,omset_usaha,usia_targetpelanggan,gender_targetpelanggan,pekerjaan_targetpelanggan,status_targetpelanggan,jenis_lokasi_
116,50030142,Usaha Kerajinan Tangan,3.0,4.0,Industri kerajinan tangan dan souvenir,NaN,"[18-39 tahun, 40-59 tahun, >60 tahun]","[Laki-laki, Perempuan]","[Pelajar/mahasiswa, \nPegawai negeri sipil, \n...","[Kelas atas, Kelas menengah]","[ Desa dan Alam, Area Wisata, Permukiman pen..."
16,30020106,Kafetaria,NaN,5.0,Industri makanan dan minuman,3.0,[18-39 tahun],"[Laki-laki, Perempuan]","[Pelajar/mahasiswa, \nPegawai negeri sipil, \n...",[Kelas Menengah],"[Pusat Kota dan Keramaian, Permukiman pendudu..."
172,60030164,Usaha Sewa Tempat Olahraga,3.0,NaN,Pendidikan dan pelatihan,3.0,"[12-17 tahun, 18-39 tahun, 40-59 tahun]","[Laki-laki, Perempuan]","[Pelajar/mahasiswa, \nPegawai negeri sipil, \n...","[Kelas atas, Kelas menengah]","[Pusat Kota dan Keramaian, Permukiman pendudu..."


Feature One Hot Encoding 

In [ ]:
#import pandas as pd
#df = pd.DataFrame({'id': [0, 1, 2], 'class': ['2 3', '1 3', '3 5']})
#df['class'] = df['class'].apply(lambda x: x.split(' '))
df_long = df.explode('usia_targetpelanggan')
df_one_hot_encoded = pd.concat([df, pd.get_dummies(df_long['usia_targetpelanggan'],prefix='is', prefix_sep='_')], axis=1)
df_one_hot_encoded_compact = df_one_hot_encoded.groupby('id_usaha').max().reset_index()

In [ ]:
df_one_hot_encoded_compact

In [ ]:
# OHE untuk feature 'bidang_usaha', 'usia_targetpelanggan', 'gender_targetpelanggan', 'pekerjaan_targetpelanggan', 'status_targetpelanggan', 'jenis_lokasi_'
prefix_bidus = pd.get_dummies(df['bidang_usaha'], prefix='is')

df = df.join(prefix_bidus)
df

In [ ]:
dum

,Industri fashion dan tekstil,Industri fashion dan tekstil,Industri kerajinan tangan dan souvenir,Industri makanan dan minuman,Jasa keamanan,Jasa kebersihan,Jasa kesehatan dan kecantikan,Jasa konsultasi dan manajemen,Jasa teknologi informasi dan komunikasi,Jasa teknologi informasi dan komunikasi,Lainnya,Otomotif,Pendidikan dan pelatihan,Perikanan dan kelautan,Pertanian dan perkebunan,Pertanian dan perkebunan
0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
179,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
180,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
181,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


### Feature Scaling

In [ ]:
# scale training data
umkm_train_unscaled = umkm_train
user_train_unscaled = user_train
y_train_unscaled    = y_train

scalerItem = StandardScaler()
scalerItem.fit(item_train)
item_train = scalerItem.transform(item_train)

scalerUser = StandardScaler()
scalerUser.fit(user_train)
user_train = scalerUser.transform(user_train)

scalerTarget = MinMaxScaler((-1, 1))
scalerTarget.fit(y_train.reshape(-1, 1))
y_train = scalerTarget.transform(y_train.reshape(-1, 1))
#ynorm_test = scalerTarget.transform(y_test.reshape(-1, 1))

print(np.allclose(item_train_unscaled, scalerItem.inverse_transform(item_train)))
print(np.allclose(user_train_unscaled, scalerUser.inverse_transform(user_train)))

## Full Scikit Learn 

In [ ]:
df = df[['id_usaha', 'jenis_usaha', 'skala_usaha', 'modal_usaha', 'bidang_usaha', 'omset_usaha', 'usia_targetpelanggan', 'gender_targetpelanggan', 'pekerjaan_targetpelanggan', 'status_targetpelanggan', 'jenis_lokasi_']]

In [ ]:
df['features'] = df['skala_usaha'] +","+ df['modal_usaha'] +","+ df['bidang_usaha'] +","+ df[ 'omset_usaha'] +","+ df['usia_targetpelanggan'] +","+ df['gender_targetpelanggan'] +","+ df['pekerjaan_targetpelanggan'] +","+ df['status_targetpelanggan'] +","+ df['jenis_lokasi_']

In [ ]:
new_df = df[['id_usaha', 'features']]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')
vectors = cv.fit_transform(new_df['features']).toarray()

In [ ]:
print(vectors[0])

[2 0 0 0 1 0 0 0 0 0 1 1 0 0 0 1 1 1 0 1 0 0 3 1 0 1 1 0 1 0 2 0 0 0 0 0 5
 0 0 1 0 0 0 1 0 0 0 0 1 1 0 2 0 1 1 1 1 0 1 0 0 0 0 1 0 1 0 0 1 0 0 0 0 1
 0 0 0 1 0 1 1 1 0 0 3 0 0 0 0 0 0 0 0 0 0]


In [ ]:
import nltk

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:
def stem(text):
    y=[]
    
    for i in text.split():
        y.append(ps.stem(i))
    
    return " ".join(y)

In [ ]:
new_df['features'] = new_df['features'].apply(stem) #applying stemming on the tags column

<ipython-input-75-067bb86c7527>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['features'] = new_df['features'].apply(stem) #applying stemming on the tags column


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity = cosine_similarity(vectors)

In [ ]:
def recommend_scikitlearn(id):
    usaha_index = new_df[new_df['id_usaha'] == id].index[0]
    distances = similarity[usaha_index]
    usaha_list = sorted(list(enumerate(distances)),reverse=True, key=lambda x:x[1])[1:7]
    
    for i in usaha_list:
        print(new_df.iloc[i[0]].id_usaha, " - ", df.iloc[i[0]].jenis_usaha)
        print(df.iloc[i[0]].modal_usaha, df.iloc[i[0]].bidang_usaha, df.iloc[i[0]].omset_usaha, df.iloc[i[0]].usia_targetpelanggan)

In [ ]:
recommend(95020152) 

95010152  -  Usaha Perbaikan Sepatu
<=3 JT Industri fashion dan tekstil  0-10 JT 2-11 tahun, 12-17 tahun, 18-39 tahun, 40-59 tahun, >60 tahun
95030152  -  Usaha Perbaikan Sepatu
5-10 JT Industri fashion dan tekstil  0-10 JT 2-11 tahun, 12-17 tahun, 18-39 tahun, 40-59 tahun, >60 tahun
50020110  -  Produksi Produk Olahan Limbah
5-10 JT Industri kerajinan tangan dan souvenir 0-10 JT 12-17 tahun, 18-39 tahun, 40-59 tahun, >60 tahun
40010153  -  Usaha Perlengkapan dan Kebutuhan Olahraga
11-30 JT Industri fashion dan tekstil 0-10 JT 12-17 tahun, 18-39 tahun, 40-59 tahun
50010109  -  Produksi Produk Olahan Limbah
<=3 JT Industri kerajinan tangan dan souvenir 0-10 JT 12-17 tahun, 18-39 tahun, 40-59 tahun, >60 tahun
30010103  -  Bisnis Makanan Frozen Food
5-10 JT Industri makanan dan minuman 0-10 JT 2-11 tahun, 12-17 tahun, 18-39 tahun, 40-59 tahun


In [ ]:
df[df['id_usaha']==95010152]

,id_usaha,jenis_usaha,skala_usaha,modal_usaha,bidang_usaha,omset_usaha,usia_targetpelanggan,gender_targetpelanggan,pekerjaan_targetpelanggan,status_targetpelanggan,jenis_lokasi_,features
142,95010152,Usaha Perbaikan Sepatu,Mikro,<=3 JT,Industri fashion dan tekstil,0-10 JT,"2-11 tahun, 12-17 tahun, 18-39 tahun, 40-59 ta...","Laki-laki, Perempuan","Pelajar/mahasiswa,\nPegawai negeri sipil,\nWir...","Kelas atas, Kelas menengah, Menuju kelas menengah","Pusat Kota dan Keramaian, Desa dan Alam, Lingk...","Mikro,<=3 JT,Industri fashion dan tekstil ,0-1..."


In [ ]:
df[df['id_usaha']==50020110]

,id_usaha,jenis_usaha,skala_usaha,modal_usaha,bidang_usaha,omset_usaha,usia_targetpelanggan,gender_targetpelanggan,pekerjaan_targetpelanggan,status_targetpelanggan,jenis_lokasi_,features
25,50020110,Produksi Produk Olahan Limbah,Kecil,5-10 JT,Industri kerajinan tangan dan souvenir,0-10 JT,"12-17 tahun, 18-39 tahun, 40-59 tahun, >60 tahun","Laki-laki, Perempuan","Pelajar/mahasiswa,\nPegawai negeri sipil,\nWir...","Kelas atas, Kelas menengah, Menuju kelas menen...","Pusat Kota dan Keramaian, Desa dan Alam, Permu...","Kecil ,5-10 JT,Industri kerajinan tangan dan s..."


## TextVectorization with Tensorflow vectorization - stemmer

In [ ]:
df = df[['user_id', 'jenis_usaha', 'skala_usaha', 'modal_usaha', 'bidang_usaha', 'omset_usaha', 'usia_targetpelanggan', 'gender_targetpelanggan', 'pekerjaan_targetpelanggan', 'status_targetpelanggan', 'jenis_lokasi_']]
df['features'] = df['skala_usaha'] +" " + df['modal_usaha'] +" "+ df['bidang_usaha'] +" "+ df[ 'omset_usaha'] +" "+ df['usia_targetpelanggan'] +" "+ df['gender_targetpelanggan'] +" "+ df['pekerjaan_targetpelanggan'] +" "+ df['status_targetpelanggan'] +" "+ df['jenis_lokasi_']

new_df = df[['user_id', 'features']]

new_df.features = new_df.features.apply(lambda x: x.replace(",","").replace("-","").replace(">","").replace("/"," ").replace("<","").replace("=",""))

<ipython-input-14-b771183caae5>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.features = new_df.features.apply(lambda x: x.replace(",","").replace("-","").replace(">","").replace("/"," ").replace("<","").replace("=",""))


In [ ]:
df['features']

0      Mikro 5-10 JT Industri makanan dan minuman 0-1...
1      Kecil  11-30 JT Industri makanan dan minuman 1...
2      Menengah 30-50 JT Industri makanan dan minuman...
3      Mikro 11-30 JT Jasa kesehatan dan kecantikan 1...
4      Kecil  30-50 JT Jasa kesehatan dan kecantikan ...
5      Menengah 50-500 JT Jasa kesehatan dan kecantik...
6      Mikro 5-10 JT Industri makanan dan minuman 0-1...
7      Kecil  11-30 JT Industri makanan dan minuman 1...
8      Menengah 30-50 JT Industri makanan dan minuman...
9      Mikro 5-10 JT Pertanian dan perkebunan 0-10 JT...
10     Kecil  11-30 JT Pertanian dan perkebunan 11-20...
11     Menengah 30-50 JT Pertanian dan perkebunan 21-...
12     Mikro 11-30 JT Industri fashion dan tekstil  2...
13     Kecil  30-50 JT Industri fashion dan tekstil  ...
14     Menengah 50-500 JT Industri fashion dan teksti...
15     Mikro 11-30 JT Industri makanan dan minuman 11...
16     Kecil  30-50 JT Industri makanan dan minuman 2...
17     Menengah 50-500 JT Indus

In [ ]:
import nltk 
from nltk.stem.porter import PorterStemmer 

ps=PorterStemmer()

def stem(txt): 
  y=[]

  for i in txt.split(): 
    y.append(ps.stem(i))

  return " ".join(y)

new_df.features = new_df.features.apply(stem)

<ipython-input-16-3dd0ab83138c>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.features = new_df.features.apply(stem)


In [ ]:
new_df['features']

0      mikro 510 jt industri makanan dan minuman 010 ...
1      kecil 1130 jt industri makanan dan minuman 112...
2      menengah 3050 jt industri makanan dan minuman ...
3      mikro 1130 jt jasa kesehatan dan kecantikan 11...
4      kecil 3050 jt jasa kesehatan dan kecantikan 31...
5      menengah 50500 jt jasa kesehatan dan kecantika...
6      mikro 510 jt industri makanan dan minuman 010 ...
7      kecil 1130 jt industri makanan dan minuman 112...
8      menengah 3050 jt industri makanan dan minuman ...
9      mikro 510 jt pertanian dan perkebunan 010 jt 1...
10     kecil 1130 jt pertanian dan perkebunan 1120 jt...
11     menengah 3050 jt pertanian dan perkebunan 2130...
12     mikro 1130 jt industri fashion dan tekstil 213...
13     kecil 3050 jt industri fashion dan tekstil 213...
14     menengah 50500 jt industri fashion dan tekstil...
15     mikro 1130 jt industri makanan dan minuman 112...
16     kecil 3050 jt industri makanan dan minuman 213...
17     menengah 50500 jt indust

In [ ]:
!pip install -upgrade tensorflow-gpu


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: -u


In [ ]:
import tensorflow as tf 
from tensorflow import keras

from tensorflow.keras.layers.experimental.preprocessing import TextVectorization 

In [ ]:
text_features= tf.data.Dataset.from_tensor_slices(new_df['features'])
max_features = 5000 
max_len = 70


In [ ]:
vectorize_layer = tf.keras.layers.TextVectorization(
    max_tokens=max_features, 
    output_mode='int', 
    output_sequence_length = max_len
)

vectorize_layer.adapt(text_features.batch(10))

model = tf.keras.models.Sequential()

model.add(tf.keras.Input(shape=(1,), dtype=tf.string))
model.add(vectorize_layer)

#new_df.features = model.predict(new_df['features'])
x =  model.predict(new_df['features'])

6/6 [==============================] - 0s 2ms/step


In [ ]:
print(new_df['features'][1])

kecil 1130 jt industri makanan dan minuman 1120 jt 1839 tahun 4059 tahun 60 tahun perempuan pelajar mahasiswa petani nelayan pekerja inform irt kela ata kela menengah menuju kela menengah kela rentan kela miskin pusat kota dan keramaian permukiman penduduk desa dan alam


In [ ]:
print(x[1])

[38 42  5 28 53  4 52 46  5  9  2 13  2 27  2 10 14 15 57 69 20 21 25  3
  8  3  6 37  3  6  3 47  3 48  7 11  4 12 24 23 49  4 45  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(x)

In [ ]:
def recommend(id):
    usaha_index = new_df[new_df['user_id'] == id].index[0]
    distances = similarity[usaha_index]
    usaha_list = sorted(list(enumerate(distances)),reverse=True, key=lambda x:x[1])[:5]
    
    for i in usaha_list:
        print(new_df.iloc[i[0]].user_id, " - ", df.iloc[i[0]].jenis_usaha)
        print(i)
        print(df.iloc[i[0]].modal_usaha, df.iloc[i[0]].bidang_usaha, df.iloc[i[0]].omset_usaha, df.iloc[i[0]].usia_targetpelanggan, df.iloc[i[0]].gender_targetpelanggan) #df.iloc[i[0]].pekerjaan_targetpelanggan,df.iloc[i[0]].jenis_lokasi_ )

In [ ]:
recommend(70030144) 

50030145  -  Usaha Panglong Kayu
(121, 1.0000000000000004)
50-500 JT Industri kerajinan tangan dan souvenir 21-30 JT 18-39 tahun, 40-59 tahun, >60 tahun Laki-laki
50030145  -  Usaha Panglong Kayu
(122, 0.919786258423082)
<=1 M Industri kerajinan tangan dan souvenir >50 JT  18-39 tahun, 40-59 tahun, >60 tahun Laki-laki
50020133  -  Usaha Art Lukisan
(88, 0.8156983121839398)
11-30 JT Industri kerajinan tangan dan souvenir 11-20 JT  18-39 tahun, 40-59 tahun, >60 tahun Laki-laki, Perempuan
50030133  -  Usaha Art Lukisan
(89, 0.8028745598743883)
30-50 JT Industri kerajinan tangan dan souvenir 21-30 JT 18-39 tahun, 40-59 tahun, >60 tahun Laki-laki, Perempuan
50010133  -  Usaha Art Lukisan
(87, 0.798278035584891)
3-5 JT Industri kerajinan tangan dan souvenir 0-10 JT 18-39 tahun, 40-59 tahun, >60 tahun Laki-laki, Perempuan


## Function Modelling

In [ ]:
import pandas as pd 
import numpy as np 

import tensorflow as tf 
from tensorflow import keras
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization 

import nltk 
from nltk.stem.porter import PorterStemmer 

from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
def ETL(csv_file):
    """
    input:
        csv file
    deskripsi:
        mengekstrak, transformasi data, 
        dan load data kedalam dataframe/variabel baru
    output:
        df,new_df,users
    """
    #Variabel Lokal
    users={}

    #Extract
    df = pd.read_csv(csv_file)
    users = extract_user_datas(df)

    # print(df)
    # df.drop_duplicates(inplace=True)
    # df.info()
    # df.isnull().sum()
    # df.duplicated().sum()

    # developer-friendly columns name
    df.columns = [x.lower().replace(' ', '_').replace('-', '_') for x in df.columns.to_list()]

    #Transform
    df.user_id= df.user_id.astype(str)

    #Load
    #load main datas to new dataframe will be used
    df = df[['user_id', 'jenis_usaha', 'skala_usaha', 'modal_usaha', 'bidang_usaha', 'omset_usaha', 'usia_targetpelanggan', 'gender_targetpelanggan', 'pekerjaan_targetpelanggan', 'status_targetpelanggan', 'jenis_lokasi_']]
    df['features'] = df['skala_usaha'] +" " + df['modal_usaha'] +" "+ df['bidang_usaha'] +" "+ df[ 'omset_usaha'] +" "+ df['usia_targetpelanggan'] +" "+ df['gender_targetpelanggan'] +" "+ df['pekerjaan_targetpelanggan'] +" "+ df['status_targetpelanggan'] +" "+ df['jenis_lokasi_']
    new_df = df[["user_id","features"]]
    new_df.features = new_df.features.apply(lambda x: x.replace(",","").replace("-","").replace(">","").replace("/"," "))
    new_df.features = new_df.features.apply(lambda x: x.lower())

    # new_df

    return df,new_df,users

def extract_user_datas(df):
    """
    output: 
        list users
    """
    row = df.values.tolist()
    # print(row[0])
    for r in row:
        # print(r)
        user_meta_data = {
            'user_id': r[0], 
            'jenis_usaha':r[1], 
            'skala_usaha':r[2], 
            'modal_usaha':r[3],
            'bidang_usaha':r[4],
            'omset_usaha':r[5], 
            'usia_targetpelanggan':r[6], 
            'gender_targetpelanggan':r[7], 
            'pekerjaan_targetpelanggan':r[8], 
            'status_targetpelanggan':r[9], 
            'jenis_lokasi_':r[10]
        }

        users[r[0]] = user_meta_data
    return users

In [ ]:
users = {}
user_meta_data = {
    'user_id': '', 
    'jenis_usaha':'', 
    'skala_usaha':'', 
    'modal_usaha':'',
    'bidang_usaha':'',
    'omset_usaha':'', 
    'usia_targetpelanggan':'', 
    'gender_targetpelanggan':'', 
    'pekerjaan_targetpelanggan':'', 
    'status_targetpelanggan':'',
    'jenis_lokasi_':''
}
df,new_df,users = ETL("/content/drive/MyDrive/dummy_data - usaha.csv")

<ipython-input-10-bd62d0ae5b86>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.features = new_df.features.apply(lambda x: x.replace(",","").replace("-","").replace(">","").replace("/"," "))
<ipython-input-10-bd62d0ae5b86>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.features = new_df.features.apply(lambda x: x.lower())


In [ ]:
print(users)

{30010101: {'user_id': 30010101, 'jenis_usaha': 'Agent Air Mineral dan Gas LPJ', 'skala_usaha': 'Mikro', 'modal_usaha': '5-10 JT', 'bidang_usaha': 'Industri makanan dan minuman', 'omset_usaha': '0-10 JT', 'usia_targetpelanggan': '18-39 tahun, 40-59 tahun, >60 tahun', 'gender_targetpelanggan': 'Perempuan', 'pekerjaan_targetpelanggan': 'Pelajar/mahasiswa, Petani, Nelayan, Pekerja informal, IRT', 'status_targetpelanggan': 'Kelas atas, Kelas menengah, Menuju kelas menengah, Kelas rentan, Kelas miskin', 'jenis_lokasi_': 'Pusat Kota dan Keramaian, Permukiman penduduk, Desa dan Alam'}, 30020101: {'user_id': 30020101, 'jenis_usaha': 'Agent Air Mineral dan Gas LPJ', 'skala_usaha': 'Kecil ', 'modal_usaha': '11-30 JT', 'bidang_usaha': 'Industri makanan dan minuman', 'omset_usaha': '11-20 JT ', 'usia_targetpelanggan': '18-39 tahun, 40-59 tahun, >60 tahun', 'gender_targetpelanggan': 'Perempuan', 'pekerjaan_targetpelanggan': 'Pelajar/mahasiswa, Petani, Nelayan, Pekerja informal, IRT', 'status_target

In [ ]:
def stem(txt):
    y = []
    porterStemmer = PorterStemmer()
    for t in txt.split():
        y.append(porterStemmer.stem(t))
    return " ".join(y)
    

text_features= tf.data.Dataset.from_tensor_slices(new_df['features'])
max_features = 5000 
max_len = 70

vectorize_layer = tf.keras.layers.TextVectorization(
    max_tokens=max_features, 
    output_mode='int', 
    output_sequence_length = max_len
)

vectorize_layer.adapt(text_features.batch(10))

model = tf.keras.models.Sequential()

model.add(tf.keras.Input(shape=(1,), dtype=tf.string))
model.add(vectorize_layer)


def getSimilarityMatrix():
    new_df.features = new_df.features.apply(stem)
    x =  model.predict(new_df['features'])
    similarity = cosine_similarity(x)

    return similarity

def getRecommendation(user_id):
    usaha_index = new_df[new_df['user_id'] == user_id].index[0]
    similarity = getSimilarityMatrix()
    distances = similarity[usaha_index]
    usaha_list = sorted(list(enumerate(distances)),reverse=True, key=lambda x:x[1])[1:5]
    
    for i in usaha_list:
        print(new_df.iloc[i[0]].user_id, " - ", df.iloc[i[0]].jenis_usaha)
        print(i)
        print(df.iloc[i[0]].modal_usaha, df.iloc[i[0]].bidang_usaha, df.iloc[i[0]].omset_usaha, df.iloc[i[0]].usia_targetpelanggan, df.iloc[i[0]].gender_targetpelanggan) #df.iloc[i[0]].pekerjaan_targetpelanggan,df.iloc[i[0]].jenis_lokasi_ )


In [ ]:
getRecommendation("70030144")

<ipython-input-13-4442b5ee352f>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.features = new_df.features.apply(stem)


6/6 [==============================] - 0s 2ms/step
70020144  -  Usaha Optik dan Kesehatan Mata
(120, 0.9658865883005587)
30-50 JT Jasa kesehatan dan kecantikan 11-20 JT  2-11 tahun, 12-17 tahun, 18-39 tahun, 40-59 tahun, >60 tahun Laki-laki, Perempuan
70030102  -  Barbershop
(5, 0.9456174041655385)
50-500 JT Jasa kesehatan dan kecantikan 41-50 JT  2-11 tahun, 12-17 tahun, 18-39 tahun, 40-59 tahun, >60 tahun Laki-laki
70020102  -  Barbershop
(4, 0.9238800635786312)
30-50 JT Jasa kesehatan dan kecantikan 31-40 JT 2-11 tahun, 12-17 tahun, 18-39 tahun, 40-59 tahun, >60 tahun Laki-laki
70030114  -  Salon
(37, 0.9195392486415576)
50-500 JT Jasa kesehatan dan kecantikan 41-50 JT   2-11 tahun, 12-17 tahun, 18-39 tahun Perempuan
